In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

E: Package 'python-software-properties' has no installation candidate


In [2]:
!pip install keras-tcn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd drive


/content/drive


In [5]:
cd My Drive

/content/drive/My Drive


In [6]:
import keras
import os
os.chdir('/content/drive/My Drive/44Col/')
print(os.listdir())

Using TensorFlow backend.


['94_2', '93_2', '92_2', '98_3', '99_3', '9_0', '95_2', '91_2', '96_2', '97_3', '8_0', '88_2', '85_2', '90_2', '86_2', '83_2', '89_2', '82_2', '87_2', '84_2', '7_0', '74_2', '73_2', '80_2', '79_2', '81_2', '75_2', '77_2', '78_2', '76_2', '64_2', '72_2', '66_2', '68_2', '65_2', '71_2', '70_2', '69_2', '67_2', '6_0', '60_2', '62_2', '57_2', '61_2', '56_2', '58_2', '5_0', '55_1', '59_2', '63_2', '52_1', '53_1', '49_1', '54_1', '4_0', '48_1', '51_1', '47_1', '50_1', '46_1', '38_1', '37_1', '41_1', '43_1', '39_1', '45_1', '40_1', '3_0', '44_1', '42_1', '33_0', '35_1', '36_1', '34_0', '32_0', '2_0', '28_0', '30_0', '29_0', '31_0', '26_0', '24_0', '232_5', '22_0', '229_5', '230_5', '231_5', '27_0', '25_0', '23_0', '227_5', '228_5', '224_5', '221_5', '21_0', '226_5', '225_5', '220_5', '223_5', '222_5', '217_5', '218_5', '213_4', '219_5', '211_4', '212_4', '214_4', '216_5', '210_4', '215_5', '205_3', '209_3', '208_3', '201_3', '204_3', '207_3', '20_0', '206_3', '202_3', '203_3', '196_3', '1_0',

In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [16]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 16:23:15 2018

@author: Pooja
"""
#Meand and Cross Validation
import numpy as np
import os
import cv2
import keras
import time
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Flatten, Dropout,Input, Bidirectional,BatchNormalization
#from keras.layers import Merge
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed	
from keras.layers.convolutional import (Conv2D,MaxPooling2D)
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint	
from keras.models import load_model
from keras.layers import multiply
from keras.layers.core import *
from keras.models import *
max_seq = 44
num = np.zeros(6)
path_uiuc = '/content/drive/My Drive/'
x = list()
y = list()
data_fold = ['44D1', '44D2', '44Col']
for fold in data_fold:
    path_44 = path_uiuc + fold + '/'
    print(path_44)
    vidlist = os.listdir(path_44)
    it = len(vidlist)
    for i in range(it):
        s1 = vidlist[i]
        _, typ = s1.split('_')
        #NO REACT
        if typ == '3':
            y.append(3)
        
        elif typ == '0':
           y.append(0)
           
        elif typ == '1':
           y.append(1)
           
        elif typ == '2':
           y.append(2)
           
        elif typ == '4':   
           y.append(4)
           
        elif typ == '5':
           y.append(5)
        else:
            print('Error!' + str(typ))
            
        num[int(typ)] +=1
        
        os.chdir(path_44 + s1)   
        img_list = os.listdir()
        it0 = len(img_list)
        temp = list()
        dim = (32,32)
        for j in range(it0):
            img = cv2.imread(img_list[j])
            print(str(i)+'_'+str(j), end = ' ')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            mn1 = np.mean(img)
            st1 = np.std(img)
            img = (img.copy() - mn1) / st1
            #img = img.copy().flatten()
            temp.append(img)
        temp = np.asarray(temp)     
        x.append(temp)
        
strokes = {'backhand':num[0], 'forehand':num[1], 'lob':num[2], 'react':num[3], 'serve':num[4], 'smash':num[5]}
for i in strokes.keys():
    print("{} - %d".format(i)%(strokes[i]))
    
x = np.asarray(x)
y = np.asarray(y)
print(x.shape)
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], x.shape[3], 1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42,shuffle =True)
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
#y_train = np_utils.to_categorical(y_train, 6)
y_test = np_utils.to_categorical(y_test, 6)

seed = 10   
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
ii = 0 

#model
x = Input((44,32,32,1))
model = (TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')))(x)
model = (TimeDistributed(Conv2D(32, (3,3),  activation='relu')))(model)
model = (TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))(model)
model = (BatchNormalization())(model)
#model = (Dropout(0.25)(model)

model = (TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')))(model)
model = (TimeDistributed(Conv2D(64, (3,3),  activation='relu')))(model)
model = (TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))(model)
model = (BatchNormalization())(model)
#model = (Dropout(0.25)(model)

model = (TimeDistributed(Flatten()))(model)

model = (Dropout(0.5))(model)
model = (Bidirectional(LSTM(512, return_sequences=False, dropout=0.4)))(model)
model = (Dense(6, activation='softmax'))(model)
model = Model(input = x, output = model)
print(model.summary())
start = time.time()
for train, test in kfold.split(X_train, y_train):
    print('------------ITERATION : ' + str(ii) + '  -------------------') 
	# Compile model
    #sgd = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    sgd = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	# Fit the model
    Yy = np_utils.to_categorical(y_train, 6)
    model.fit(X_train[train], Yy[train], epochs=80, batch_size=32, verbose=1)
    model.save('/content/drive/My Drive/conv_s_m'+str(ii)+'.h5')
    ii += 1
    #print(model.predict(X[train], batch_size=32))
	# evaluate the model
    scores = model.evaluate(X_train[test], Yy[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
exectime = time.time()-start    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

final_scores = []
for i in range(1):
  mod = load_model('/content/drive/My Drive/conv_s_m'+str(i)+'.h5')
  eval1 = time.time()
  scores = model.evaluate(X_test, y_test, verbose=1)
  evaltim = time.time()-eval1
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  final_scores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(final_scores), np.std(final_scores)))
print(exectime, evaltim)

/content/drive/My Drive/44D1/
0_0 0_1 0_2 0_3 0_4 0_5 0_6 0_7 0_8 0_9 0_10 0_11 0_12 0_13 0_14 0_15 0_16 0_17 0_18 0_19 0_20 0_21 0_22 0_23 0_24 0_25 0_26 0_27 0_28 0_29 0_30 0_31 0_32 0_33 0_34 0_35 0_36 0_37 0_38 0_39 0_40 0_41 0_42 0_43 1_0 1_1 1_2 1_3 1_4 1_5 1_6 1_7 1_8 1_9 1_10 1_11 1_12 1_13 1_14 1_15 1_16 1_17 1_18 1_19 1_20 1_21 1_22 1_23 1_24 1_25 1_26 1_27 1_28 1_29 1_30 1_31 1_32 1_33 1_34 1_35 1_36 1_37 1_38 1_39 1_40 1_41 1_42 1_43 2_0 2_1 2_2 2_3 2_4 2_5 2_6 2_7 2_8 2_9 2_10 2_11 2_12 2_13 2_14 2_15 2_16 2_17 2_18 2_19 2_20 2_21 2_22 2_23 2_24 2_25 2_26 2_27 2_28 2_29 2_30 2_31 2_32 2_33 2_34 2_35 2_36 2_37 2_38 2_39 2_40 2_41 2_42 2_43 3_0 3_1 3_2 3_3 3_4 3_5 3_6 3_7 3_8 3_9 3_10 3_11 3_12 3_13 3_14 3_15 3_16 3_17 3_18 3_19 3_20 3_21 3_22 3_23 3_24 3_25 3_26 3_27 3_28 3_29 3_30 3_31 3_32 3_33 3_34 3_35 3_36 3_37 3_38 3_39 3_40 3_41 3_42 3_43 4_0 4_1 4_2 4_3 4_4 4_5 4_6 4_7 4_8 4_9 4_10 4_11 4_12 4_13 4_14 4_15 4_16 4_17 4_18 4_19 4_20 4_21 4_22 4_23 4_24 4_25 4_26 4_27 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:125: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 44, 32, 32, 1)     0         
_________________________________________________________________
time_distributed_43 (TimeDis (None, 44, 32, 32, 32)    320       
_________________________________________________________________
time_distributed_44 (TimeDis (None, 44, 30, 30, 32)    9248      
_________________________________________________________________
time_distributed_45 (TimeDis (None, 44, 15, 15, 32)    0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 44, 15, 15, 32)    128       
_________________________________________________________________
time_distributed_46 (TimeDis (None, 44, 15, 15, 64)    18496     
_________________________________________________________________
time_distributed_47 (TimeDis (None, 44, 13, 13, 64)    36928     
__________